Change the working directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Project5_AgeGenderEmotion_Detection/1.2_gender_input_output
! ls

/content/drive/My Drive/Project5_Computer_Vision_in_Retail/1.2_gender_input_output
input  output


Load necessary python libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split

Load dataset images and split them into training and test set

In [ ]:
# Ref:  https://github.com/ChibaniMohamed/cnn_age_gender/blob/main/train.py

path = './input/UTKFace'
pixels = []
age = []
gender = [] 

i=0
for img in os.listdir(path):
  i=i+1
  ages = img.split("_")[0]
  genders = img.split("_")[1]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img=cv2.resize(img,(100,100))
  pixels.append(np.array(img))
  gender.append(np.array(genders))

pixels = np.array(pixels)
gender = np.array(gender,np.uint64)

x_train,x_test,y_train,y_test = train_test_split(pixels,gender,random_state=100)

Define Model configurations

In [ ]:
input = Input(shape=(100,100,1))
conv1 = Conv2D(70,(3,3),activation="relu")(input)
conv2 = Conv2D(65,(3,3),activation="relu")(conv1)
batch1 = BatchNormalization()(conv2)
pool3 = MaxPool2D((2,2))(batch1)
conv3 = Conv2D(60,(3,3),activation="relu")(pool3)
batch2 = BatchNormalization()(conv3)
pool4 = MaxPool2D((2,2))(batch2)
flt = Flatten()(pool4)

In [ ]:
#gender
gender_l = Dense(64,activation="relu")(flt)
gender_l = Dense(40,activation="relu")(gender_l)
gender_l = Dense(32,activation="relu")(gender_l)
gender_l = Dense(16,activation="relu")(gender_l)
gender_l = Dropout(0.5)(gender_l)
gender_l = Dense(2,activation="softmax")(gender_l)

In [ ]:
model = Model(inputs=input,outputs=gender_l)
model.compile(optimizer="adam",loss=["sparse_categorical_crossentropy"],metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 98, 98, 70)        700       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 65)        41015     
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 65)        260       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 65)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 60)        35160     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 60)        240   

Fit/ Train the model

In [ ]:
save = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30)
model.save("./output/gender_model.h5")

Epoch 1/30
556/556 [==============================] - 33s 27ms/step - loss: 0.5161 - accuracy: 0.7385 - val_loss: 0.4195 - val_accuracy: 0.7958
Epoch 2/30
556/556 [==============================] - 15s 26ms/step - loss: 0.3701 - accuracy: 0.8174 - val_loss: 0.2993 - val_accuracy: 0.8610
Epoch 3/30
556/556 [==============================] - 15s 26ms/step - loss: 0.3276 - accuracy: 0.8480 - val_loss: 0.3617 - val_accuracy: 0.8478
Epoch 4/30
556/556 [==============================] - 15s 26ms/step - loss: 0.2978 - accuracy: 0.8642 - val_loss: 0.4320 - val_accuracy: 0.7319
Epoch 5/30
556/556 [==============================] - 15s 26ms/step - loss: 0.2734 - accuracy: 0.8791 - val_loss: 0.4488 - val_accuracy: 0.8024
Epoch 6/30
556/556 [==============================] - 15s 26ms/step - loss: 0.2514 - accuracy: 0.8868 - val_loss: 0.3796 - val_accuracy: 0.8549
Epoch 7/30
556/556 [==============================] - 15s 26ms/step - loss: 0.2321 - accuracy: 0.9028 - val_loss: 0.2885 - val_accuracy:

**SAMPLE** OUTPUT OF TRAINING

In [ ]:
# Epoch 1/5
# 556/556 [==============================] - 2021s 4s/step - loss: 0.6677 - accuracy: 0.5869 - val_loss: 0.3986 - val_accuracy: 0.8223
# Epoch 2/5
# 556/556 [==============================] - 2001s 4s/step - loss: 0.4717 - accuracy: 0.7626 - val_loss: 0.4147 - val_accuracy: 0.8223
# Epoch 3/5
# 556/556 [==============================] - 2013s 4s/step - loss: 0.3846 - accuracy: 0.8170 - val_loss: 0.3072 - val_accuracy: 0.8512
# Epoch 4/5
# 556/556 [==============================] - 1989s 4s/step - loss: 0.3158 - accuracy: 0.8456 - val_loss: 0.3685 - val_accuracy: 0.8353
# Epoch 5/5
# 556/556 [==============================] - 1985s 4s/step - loss: 0.2810 - accuracy: 0.8700 - val_loss: 0.3248 - val_accuracy: 0.8645